# FIFA
### Arnold Ballesteros

Objective: Acquire Top Strikers and collect data


In [6]:
import pandas as pd
import requests
from pytrends.request import TrendReq
import pygsheets
from bs4 import BeautifulSoup as bs
import pprint

###### Grab Top players through the website: Transfermarkt.com
- headers is used to tell request that we are accessing it through a browser, otherwise access is declined
- player_info[0] contains all player names
- player_info[1] contains all ids corresponding to the player

In [7]:
url='http://www.transfermarkt.com/detailsuche/spielerdetail/suche/9069756'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
player_info = [[] for i in range(2)]
player_name=[]
response=requests.get(url,headers=headers)
#response.status_code
soup = bs(response.content,"html.parser")
table = soup.find(class_='items')
for item in table.find_all('a',{'class':'spielprofil_tooltip'}):
    player_name.append(str(item.get('title')))
    #print(item.get('title') + ':' + item.get('id'))
    player_info[0].append(item.get('title').replace(' ','-'))
    player_info[1].append(item.get('id'))

In [8]:
#list_divider = lambda list_name,chunk_size: [list_name[i:i+chunk_size] for i in range(0, len(list_name), chunk_size)]

#kw_list=list_divider(player_name,5)

#print(kw_list[1])

list_df = [[] for i in range(len(player_name))]
for name in player_name:
   
    pytrend=TrendReq()
    pytrend.build_payload([name])
    
    interest_over_time_df = pytrend.interest_over_time()
    time = pd.to_datetime(interest_over_time_df.index)
    
    interest_over_time_df['date'] = time.strftime("%m-%d-%Y")
    #interest_over_time_df['name'] = name
    interest_over_time_df = interest_over_time_df.loc[interest_over_time_df['isPartial'] == 'False']
    interest_over_time_df.drop(['isPartial'],inplace=True,axis=1)
  
    interest_df = pd.melt(interest_over_time_df, id_vars=['date'], var_name='Player',value_name='Hits')
    list_df.append(interest_df)

    #print(interest_df)
    #print(inters)
del list_df[0:25]
trend_df = pd.concat(list_df)

In [10]:
frames={}
player_headers = ["Player"]

data = [[] for i in range(19)]
head=0
for x in range(len(player_info[0])):
    #player_url= "https://www.transfermarkt.com/"+ str(player_info[0][x])+ "/leistungsdatendetails/spieler/" + str(player_info[1][x])
    player_response = requests.get("https://www.transfermarkt.com/"+ str(player_info[0][x])+ "/leistungsdatendetails/spieler/" + str(player_info[1][x]) + "/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1", headers = headers )
    player_soup = bs(player_response.content,"html.parser")
    
    table = player_soup.find(class_='items')

    if head == 0:
        for item in table.find_all('a',{'class':'sort-link'}):
        #print(item.contents[0])
            if str(type(item.contents[0])) == "<class 'bs4.element.NavigableString'>":
                player_headers.append(item.contents[0])
            else:
                player_headers.append(item.contents[0].get('title'))
        head =1
    

    table_body = table.find('tbody')
    rows = table_body.find_all('tr')

    for row in rows:
        i=0
        
        data[i].append(player_info[0][x])
        i=i+1
        for element in row.find_all('td'):
            #print(i)
            if (str(element.contents[0])[0]) == "<":
                if str(type(element.find('a'))) == "<class 'NoneType'>":
                    1+1
                else:
                    if str(element.find('a').contents[0])[0]=="<":
                    
                        data[i].append(str(element.find('a').get('href')).strip('/').split('/')[0])
                    else:
                        if str(element.contents[0]) == "-":
                            data[i].append(0)
                        else:
                            data[i].append(element.find('a').contents[0])
            else:
                if str(element.contents[0]) == "-":
                    data[i].append(0)
                else:
                    data[i].append(element.contents[0])
            i=i+1
print(data)        
data.pop(2)
player_headers.pop(3)
player_df = pd.DataFrame(data)
player_df = player_df.transpose()
player_df.columns = player_headers
frames[player_info[0][x]]=player_df

[['Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Neymar', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi', 'Lionel-Messi'

In [11]:

print(player_df)

             Player Season                     Competition  \
0            Neymar  17/18                         Ligue 1   
1            Neymar  17/18                Champions League   
2            Neymar  17/18               Coupe de la Ligue   
3            Neymar  17/18                 Coupe de France   
4            Neymar  16/17                    Copa del Rey   
5            Neymar  16/17                          LaLiga   
6            Neymar  16/17                Champions League   
7            Neymar  15/16                    Copa del Rey   
8            Neymar  15/16                          LaLiga   
9            Neymar  15/16                Champions League   
10           Neymar  15/16                  Club World Cup   
11           Neymar  14/15                Champions League   
12           Neymar  14/15                    Copa del Rey   
13           Neymar  14/15                          LaLiga   
14           Neymar  13/14                          LaLiga   
15      

In [151]:
gc = pygsheets.authorize(outh_file="D:\Documents\Projects\FIFA\client_secret.json")

sh=gc.open('Top FIFA Strikers - Data')
sh.worksheet_by_title("Player Data").set_dataframe(player_df,(1,1))

sh=gc.open('Top FIFA Strikers - Trends')
sh.worksheet_by_title("Trends").set_dataframe(trend_df,(1,1))

sh=gc.open('Top FIFA Strikers')
sh.worksheet_by_title("Trends").set_dataframe(trend_df,(1,1))
sh.worksheet_by_title("Player Data").set_dataframe(player_df,(1,1))
